In [1]:
import numpy as np    
import pandas as pd   
# import matplotlib.pyplot as plt

In [155]:
df = pd.read_csv('breast-cancer.csv')
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [156]:
df_x = df.drop(columns=['diagnosis'])
df_y = df['diagnosis']

In [157]:
numeric_columns = df_x.select_dtypes(include=[np.number]).columns
categorical_columns = df_x.select_dtypes(include=['object','category']) #object is for string , category is for pandas object 

In [158]:
categorical_columns_with_more_than_3_unique_values = []
categorical_columns_with_till_3_unique_values = []

for i in categorical_columns: 
    if df[i].nunique()>3: 
        categorical_columns_with_more_than_3_unique_values.append(i)  
    else: 
        categorical_columns_with_till_3_unique_values.append(i) 

print(categorical_columns_with_more_than_3_unique_values)
print(categorical_columns_with_till_3_unique_values)

df_x.drop(columns=categorical_columns_with_more_than_3_unique_values, inplace=True)

[]
[]


In [159]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape 

((455, 31), (114, 31), (455,), (114,))

ordinal = rank matter     be, phd, 12th             (ordinal encoder)
nominal = rank not matter                 male, female   (labelEncoder)

one not encoding for multivariable columns 
for now we are just dropping the columns with more than 3 variable 


In [160]:
from sklearn.compose import ColumnTransformer, make_column_transformer  #takes list of tuples 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split 

ohe = OneHotEncoder(drop='first', sparse_output=False)
le = LabelEncoder()
oe = OrdinalEncoder()
si_mean = SimpleImputer(strategy='mean')
si_mf = SimpleImputer(strategy='most_frequent')
mms = MinMaxScaler()

steps_categorical = [
            ('imputer_mf', si_mf),
            ('ohe', ohe),     
]

steps_numerical = [
            ('imputer_mean', si_mean),
            ('mms', mms)
]

In [161]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression()
svc = SVC()
rf = RandomForestClassifier()

models = [lr, svc, rf]

In [162]:
from sklearn.pipeline import Pipeline

pipe_numerical = Pipeline(steps_numerical)
pipe_categorical = Pipeline(steps_categorical)


In [163]:
ct = ColumnTransformer(
        [   
            ('pipe_numerical', pipe_numerical, numeric_columns),
            ('pipe_categorical', pipe_categorical, categorical_columns_with_till_3_unique_values)
        ],
        remainder='passthrough'
    )
ct.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipe_numerical',
                                 Pipeline(steps=[('imputer_mean',
                                                  SimpleImputer()),
                                                 ('mms', MinMaxScaler())]),
                                 Index(['id', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'tex...
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')),
                                ('pipe_categorical',
                                 Pipeline(steps=[('imputer_mf',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False))]),
                                 [])])

In [164]:
steps_ct_model = Pipeline([
    ('ct', ct),
    ('model', rf)
])
steps_ct_model

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipe_numerical',
                                                  Pipeline(steps=[('imputer_mean',
                                                                   SimpleImputer()),
                                                                  ('mms',
                                                                   MinMaxScaler())]),
                                                  Index(['id', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_...
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')),
                                                 ('pipe_categorical',
                                                  Pipeline(steps=[('imputer_mf',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False))]),
                                                  [])])),
                ('model', RandomForestClassifier())])

In [165]:
rf_prams = {
    'model__criterion' : ['gini', 'entropy', 'log_loss'],
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5, 10],
    'model__max_features': ['auto', 'sqrt', 'log2']
}

svc_params = {
    'model__C': [0.1, 1, 10, 100],
    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'model__degree': [2, 3, 4],
    'model__gamma': ['scale', 'auto'],
    'model__coef0': [0.0, 0.1, 0.5, 1.0],
    'shrinking': [True, False]
}

lr_params = {
    'model__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'model__C': [0.1, 1, 10, 100],
    'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'model__max_iter': [100, 200, 300],
    'model__l1_ratio': [None, 0.1, 0.5, 0.7]  
}

In [166]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

gs = GridSearchCV(steps_ct_model, param_grid= rf_prams, n_jobs=-1)
gs.fit(x_train, y_train)
gs.best_params_, gs.best_score_

c:\Users\pubg3\.conda\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
130 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\pubg3\.conda\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\pubg3\.conda\envs\tf\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\pubg3\.conda\envs\tf\lib\site-packages\sklearn\pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params

({'model__criterion': 'log_loss',
  'model__max_depth': 10,
  'model__max_features': 'log2',
  'model__min_samples_split': 5,
  'model__n_estimators': 100},
 0.9692307692307691)

In [167]:
from sklearn.metrics import accuracy_score

gs_pred = gs.best_estimator_.predict(x_test)
accuracy_score(y_test, gs_pred)

0.9649122807017544